In [1]:
import sys
sys.path.append('../')
import lemur

In [ ]:
a = lemur.linspace(start=0, end=1, steps=20, requires_grad=True)

In [ ]:
b = a.view(lemur.tensor([1,1,2,2,5])).permute(lemur.tensor([0,1,4,2,3])).sigmoid()
#c = b.sum(lemur.tensor([0,0,0,0,0]))
#c.backward()


: 

In [11]:
a.grad

tensor([[[[[0.2500, 0.2498, 0.2493, 0.2484, 0.2473, 0.2457, 0.2439, 0.2417, 0.2392, 0.2365, 0.2335, 0.2302, 0.2266, 0.2229, 0.2189, 0.2148, 0.2104, 0.2060, 0.2013, 0.1966]]]]], shape=[1, 1, 1, 1, 20])


In [3]:
lemur.set_sci_print(False)
a.grad

tensor([[[[[0.2500, 0.2498, 0.2493, 0.2484, 0.2473, 0.2457, 0.2439, 0.2417, 0.2392, 0.2365, 0.2335, 0.2302, 0.2266, 0.2229, 0.2189, 0.2148, 0.2104, 0.2060, 0.2013, 0.1966]]]]], shape=[1, 1, 1, 1, 20])


In [14]:
import torch
a = torch.linspace(start=0, end=1, steps=20, requires_grad=True)

In [15]:
%%timeit
b = a.view(1,1,2,2,5).permute([0,1,4,2,3]).sigmoid()
c = b.sum()
c.backward()

24.6 µs ± 846 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
a = lemur.linspace(-1,1, 10, requires_grad=True)
b = a.sigmoid().sigmoid()
c = b.sum(lemur.tensor([0,0,0,0,0]))
c.backward()

In [13]:
c.graph

self
└── LemurTensor @ 0x11711eb60 (shape=[1, 1, 1, 1, 1])
    └── [sum]
        ├── LemurTensor @ 0x105823f30 (shape=[1, 1, 1, 1, 10])
        │   └── [sigmoid]
        │       └── LemurTensor @ 0x107bad330 (shape=[1, 1, 1, 1, 10])
        │           └── [sigmoid]
        │               └── LemurTensor @ 0x105819530 (shape=[1, 1, 1, 1, 10])
        └── LemurTensor @ 0x11711e7e0 (shape=[1, 1, 1, 1, 5])


In [11]:
import torch
a = torch.linspace(-1,1, 10, requires_grad=True)
b = a.sigmoid().sigmoid()
c = b.sum()
c.backward()
a.grad

tensor([0.0483, 0.0526, 0.0560, 0.0582, 0.0590, 0.0582, 0.0559, 0.0524, 0.0481,
        0.0431])

In [5]:
a = lemur.full((2,1,1,1,1),100.0, requires_grad=True)
a

tensor([[[[[100.00]]]],



	[[[[100.00]]]]], shape=[2, 1, 1, 1, 1])

In [7]:
lemur.LEMUR_SCI_PRINT = False
a

tensor @ 0x14478dab0
comes_from: [NULL expression]
k:
kernel_tensor @ 0x1150c2f80 with length = 4 
stride = [4, 4, 4, 4, 1]
computed = false
shallow = false
tensor([[[[[0.00e+00, 1.00e+00, 2.00e+00, 3.00e+00]]]]])
shape = [1, 1, 1, 1, 4]
grad:
kernel_tensor @ 0x1147f4ba0 with length = 4 
stride = [4, 4, 4, 4, 1]
computed = false
shallow = false
tensor([[[[[0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00]]]]])
shape = [1, 1, 1, 1, 4]

In [3]:
lemur.full((1,1,1,1,1),100.0)

tensor @ 0x139fcf5c0
comes_from: [NULL expression]
k:
kernel_tensor @ 0x139fd6670 with length = 1 
stride = [1, 1, 1, 1, 1]
computed = false
shallow = false
tensor([[[[[1.00e+02]]]]])
shape = [1, 1, 1, 1, 1]
grad:
[NULL kernel_tensor]

In [ ]:
b = a.view(lemur.tensor([1,1,1,2,2]))
c = b.permute(lemur.tensor([0,1,2,4,3])).sigmoid()
d = c.sum(lemur.tensor([0,0,0,0,0]))
d.backward()


In [4]:
print(d.graph())

└── LemurTensor @ 0x13e16cc50 
    └── [sum]
        ├── LemurTensor @ 0x13e135d20 (shape=[4, 2, 4])
        │   └── [sigmoid]
        │       └── LemurTensor @ 0x13e13a960 (shape=[4, 2, 4])
        │           └── [permute]
        │               ├── LemurTensor @ 0x13c6a1c10 (shape=[2, 4, 4])
        │               │   └── [view]
        │               │       ├── LemurTensor @ 0x13e15d950 (shape=[32])
        │               │       └── LemurTensor @ 0x13c69fe00 (shape=[5])
        │               └── LemurTensor @ 0x13e152480 (shape=[5])
        └── LemurTensor @ 0x13e1231c0 (shape=[5])


In [4]:
a

tensor @ 0x13a052840
comes_from: [NULL expression]
k:
kernel_tensor @ 0x11941f600 with length = 4 
stride = [4, 4, 4, 4, 1]
computed = false
shallow = false
tensor([[[[[0.00e+00, 1.00e+00, 2.00e+00, 3.00e+00]]]]])
shape = [1, 1, 1, 1, 4]
grad:
kernel_tensor @ 0x13a042db0 with length = 4 
stride = [4, 4, 4, 4, 1]
computed = false
shallow = false
tensor([[[[[2.50e-01, 1.97e-01, 1.05e-01, 4.52e-02]]]]])
shape = [1, 1, 1, 1, 4]

In [31]:
import torch

In [44]:
a = torch.arange(4., requires_grad=True, dtype=torch.float32)
b = a.view(1,1,1,2,2).permute(0,1,2,4,3)
c = b.sigmoid()
d = c.sum()
d.backward()
a.grad

tensor([0.2500, 0.1966, 0.1050, 0.0452])

In [46]:
#a = torch.arange(32., requires_grad=True, dtype=torch.float32)
a = torch.tensor([[[[[0.,1],[2,3]]]]], requires_grad=True)
b = a.view(1,1,1,2,2).permute(0,1,2,4,3)
c = b.sigmoid()
d = c.sum()
d.backward()
a.grad

tensor([[[[[0.2500, 0.1966],
           [0.1050, 0.0452]]]]])

In [11]:
tensor([2.5000e-01, 1.9661e-01, 1.0499e-01, 4.5177e-02, 1.7663e-02, 6.6480e-03,
        2.4665e-03, 9.1017e-04, 3.3522e-04, 1.2337e-04, 4.5417e-05, 1.6689e-05,
        6.1988e-06, 2.2650e-06, 8.3446e-07, 3.5763e-07, 1.1921e-07, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00])

NameError: name 'tensor' is not defined

In [7]:
a = lemur.LemurTensor(s, requires_grad=True)
b = lemur.LemurTensor(s, requires_grad=True)
c = lemur.LemurTensor(s)
r = lemur.tensor((0,0,0,0,0))

In [ ]:
lemur.tensor((0,0.5,0,0,100))

In [ ]:
d = (a * b) + c
e = (d * d) + d
f = d + e
g = f.sum(r) 
print(g.graph())

In [7]:
#%%timeit
d = (a * b) + c
e = (d * d) + d
f = a + b + c + d + e
g = f.sum(r) #fwd takes 5ms
g.backward() #backward takes 15ms

In [5]:
#d.graph()

In [15]:
import torch

In [ ]:
at = torch.arange(32., requires_grad=True)
a = at.view(2,4,4)
a.sum().backward()
at.grad


In [ ]:
at = torch.empty(s, requires_grad=True)
bt = torch.empty(s, requires_grad=True)
ct = torch.empty(s)


In [ ]:
%%timeit
dt = (at * bt) + ct
et = (dt * dt) + dt
ft = at + bt + ct + dt + et
gt = ft.sum() #fw 3ms
gt.backward() #backward 5ms

In [ ]:
a = lemur.tensor([i for i in range(1,9)], (1,1,2,2,2),  requires_grad=True)
b = lemur.tensor([i for i in range(1+10,9+10)], (1,1,2,2,2),  requires_grad=True)
dm = lemur.tensor([0,0,0,0,0])
v = lemur.tensor([0,1,2,4,3])
a_p = a.permute(v)


In [ ]:
%%timeit
c = b * a_p
d = c.sum(dm)
d.backward()


In [ ]:
a = lemur.tensor([i for i in range(1,9)], (1,1,2,2,2),  requires_grad=True)
b = lemur.tensor([i for i in range(1+10,9+10)], (1,1,2,2,2),  requires_grad=True)
dm = lemur.tensor([0,0,0,0,0])
v = lemur.tensor([0,1,2,4,3])
a_p = a.permute(v)


In [ ]:
a_p = a.permute(v)
print(a)


In [ ]:
%%timeit
at.permute(0,1,2,4,3).contiguous()

In [4]:
import torch
at = torch.tensor([[[[[1.0000, 2.0000],[3.0000, 4.0000]],[[5.0000, 6.0000],[7.0000, 8.0000]]]]],  requires_grad=True, dtype=float)
bt = torch.tensor([[[[[11.0000, 12.0000],[13.0000, 14.0000]],[[15.0000, 16.0000],[17.0000, 18.0000]]]]],  requires_grad=True, dtype=float)
at_p = at.permute([0,1,2,4,3])


In [ ]:
%%timeit
ct = bt * at_p
dt = ct.sum()
dt.backward(retain_graph=True)


In [ ]:
bt.grad

In [ ]:
x = lemur.tensor([i for i in range(1,33)], (2,2,2,2,2),  requires_grad=True)
y = x.sum(lemur.tensor([0,0,0,0,0]))
y

In [ ]:
y.backward()

In [ ]:
x = lemur.tensor([1.0], requires_grad=True)
y = lemur.tensor([4.0], requires_grad=True)

z = x + y  
w = z * x  
w = w.relu() 

w.backward() 
print("Result from main:", w)

In [36]:
shape = [1,1,1,1,10]

In [ ]:
%%timeit
x = lemur.LemurTensor(shape=shape, requires_grad=False)
y = lemur.LemurTensor(shape=shape, requires_grad=False)
x + y  

In [3]:
import torch 

In [ ]:
%%timeit
x = torch.empty(shape)
y = torch.empty(shape)
x + y  

In [ ]:
t0 = torch.tensor([10.], requires_grad=True)
t1 = torch.tensor([12.], requires_grad=True)
t2 = torch.tensor([-15.], requires_grad=True)
t4 = t0 * t1 + t2
t4


In [12]:
t4.backward()

In [ ]:
t0.grad, t1.grad, t2.grad

In [ ]:
a = lemur.tensor([[10.,12.],[-1.,-4.]], requires_grad = True)
b = lemur.rand_like(a)
c = (a @ b).mean()
c.backward()
a.grad

In [ ]:
c

In [ ]:
a = lemur.tensor([...], requires_grad = True)
b = lemur.tensor([...], requires_grad = True)
c = (a + b).relu()

In [ ]:
x = torch.tensor([[1], [2], [3]])
x.size(), x.stride()

In [ ]:
x.expand(3, 4)


In [ ]:
x.retain_grad()
x.grad

In [ ]:
x = torch.tensor([[1],[2]]).expand((2,10))
x.shape, x.stride()

In [ ]:
y = x.reshape(20)

In [ ]:
y.shape, y.stride

In [ ]:
x = torch.tensor([[10.]]).expand(1,10)
x.requires_grad = True
x.stride()

In [ ]:
x.sum().backward()
x.grad.stride()

In [99]:
x = torch.rand((3,4)).t() #(1,4)
x.requires_grad = True


In [100]:
x.sum().backward()

In [ ]:
x.grad.stride()

In [83]:
float arr[27]
shape (3,3,3) 
stride (9,3,1)

In [ ]:
t0 -> 1,1 
   -> length = 1

t1 -> 1,10 
   -> length = 1   
   -> comesfrom -> t0, expand, 1 10






In [ ]:
x.stride()

In [3]:
import torch

In [ ]:
x = torch.randn((3,1), requires_grad=True)
x2 = torch.randn((3,1), requires_grad=True)

x.stride()

In [ ]:
y = (x*x2).expand(3,4)
y.stride()

In [154]:
y.sum().backward()

In [ ]:
x.grad, x2*4

In [ ]:
torch.empty((1,2,3,4,5)).permute(1,2,3,4,0).stride()


In [ ]:
x = torch.tensor([[1]]).expand(1,10)
x.stride()


In [ ]:
y = x.relu()
y.stride()

In [ ]:
x

In [ ]:
x = torch.tensor([[0.,0.]]).permute(1,0)
x.requires_grad = True
x, x.stride(), x.is_contiguous()


In [217]:
x.sum().backward()

In [ ]:
x.grad.stride(), x.stride()

In [ ]:
x.is_contiguous()

In [ ]:
x.contiguous().stride()

In [ ]:
a = torch.arange(8).view(1,4,2)
a.stride()

In [ ]:
a.permute(1,0).is_contiguous()

In [ ]:
a = torch.arange(1,33, dtype=float, requires_grad=True)
a.view(2,2,2,2,2).sum().backward()
a.grad

In [ ]:
x = lemur.tensor([i for i in range(1,33)], (2,2,2,2,2),  requires_grad=True)
y = x.sum(lemur.tensor([0,0,0,0,0]))
y


In [ ]:
y.backward()

In [ ]:
x

In [ ]:
a.sum(0,keepdim=True).sum(1,keepdim=True).sum(2,keepdim=True).sum(3,keepdim=True)


In [ ]:
a.sum(0,keepdim=True).sum(1,keepdim=True).sum(3,keepdim=True).sum(4,keepdim=True)

In [ ]:
(2,2,2,2,2) -> (1,2,2,2,1)

In [43]:
s = (5,5,50,5,5)
a_t = torch.empty(s)
a_l = lemur.LemurTensor(shape=s, requires_grad=False)



In [ ]:
%%timeit
b = a_t.sum(0,keepdim=True).sum(1,keepdim=True).sum(2,keepdim=True).sum(3,keepdim=True)

In [ ]:
%%timeit
b = a_l.sum(lemur.tensor([0,0,0,0,0])) 

In [ ]:
import torch
torch.tensor([1,2]).expand(10)

In [ ]:
print(plot_tensor_graph(d))